---
layout: post
title:  "Convolutional Neural Network"
date:   2023-03-09 10:14:54 +0700
categories: jekyll update
---

# TOC

- [Definition](#define)
- [Code example](#code)


# Definition <a name="define"></a>

Remember the dense neural net as a linear combination of input features plus a nonlinear transformation (an activation function). For this kind of neural net, we need to flatten the input into a one-dimensional array. One thing about one dimensional array is that it doesn't imply 2-dimensional quality. Meanwhile there are a lot of data in this world that inherently has a structure of more than one dimension. For example, a colored image has 3 channels, each channel has a single-colored image with width w x height h (pixels). So this image's matrix interpretation would be a matrix of 3 dimensions: 3 w * h matrices. Moreover, if we flatten this image out to feed into a dense network like we did before, the amount of connection and parameters can become millions quite fast. So the convolution neural net (CNN) has much less parameters given the same amount of neurons and hidden layers, making it easier to train and compute.


## Convo layer

To be able to process such complex data, we use convolutional neural network. We would then use matrix multiplication to transform the input (or convolute). The function that we would use to transform those input is called a filter or kernel, effectively a matrix of small size (usually 3x3). The filter is full of weights ($$ \theta $$) as we know and it would transform the entire input matrix by sliding from top left to the bottom right horizontally. The sliding can take one, two or more values per step (which is called stride), hence it reduces the original input's data. Since sliding with a matrix loses some values untransformed at the borders, we pad the original input matrix with 0 around it so that the algorithm leaves nothing untouched. The result after being slided would be added a bias ($$ \theta_0 $$) and then activated. This is just like machine learning models as we know it, though I figured the bias addition and activation could be done after the pooling layer explained below. Each filter would then create an output called an activation map. Or we can say that it contribute to one depth of it. A convolution layer with 10 filters (kernels) would result in an activation map with depth of 10. Each depth carries a simple information about the original input, as we explain shortly below.

Those filters at the beginning layers of the neural net are dubbed low-level neurons because it resembles the mechanism of biological simple neurons (or locally receptive neurons): in the brain, there would be simple neurons that are in charge of identifying very simple shapes such as a horizontal line or a vertical line. Those simple signals then would be sent to groups of more complex neurons and those group would be able to combine the lines  to recognize more abstract patterns (corners, wrinkles, eyes, noses, etc).

Those filters can also be seen as the visual perception of the neurons in the activation maps. The perception is small, each of those neurons would be connected not to all but to a limited part of the input hence the word locally receptive. We end up with one neuron per pixel and lots of activation maps for each layer. 

## Pooling layer

After being convoluted, we can add a pooling layer to the data. A pooling matrix is usually 2x2 (up to 5x5) and it works by choosing one of the value in that window, mostly the max or sometimes the average. This means to reduce the matrix size, hence reduce complexity, computation and overfit. The depth of the activation map would be the same though. Gradient descent flows through this pooling normally (through the max data value).


## Dropout layer

Since the dimensions can grow big, there is a technique called dropout that temporarily blank out a percentage of neurons. Think of this analogy: in a company, when a person is absent, others would learn to take over his job. In general, with regular absence, the employees become more generalized in their skills and this overall is better for the company. Also for the neural net, since it becomes less overfit.

# Code example <a name="code"></a>

In this example, let's explore the fashion MNIST (similar to MNIST but for fashion images). First, let's add an extra dimension for the data, it would store the number of data points we have and together we have a tensor (multidimensional vector) for the training.

In [ ]:
mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_train_full = np.expand_dims(X_train_full, axis=-1).astype(np.float32) / 255
X_test = np.expand_dims(X_test.astype(np.float32), axis=-1) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

A CNN architecture is more complicated than a dense net. Firstly, we put a convolution layer (usually with relu activation), then a max pool, then another convolution layer (can be with double neurons, to process information better), another max pool, and so on. After that, when we are done with the convoluted layers, when we feel like the net is enough to comprehend complex feature of the data, we move on to the dense layers. Before doing that, we need to add a flatten method which is not a layer, it is just an unwinding of neurons from the convolution layer. Then we put some dense layer, mix with dropout (rate typically 0.2 to 0.5). The last layer, as usual, is a softmax for the classification problem, and relu for regression.

In [ ]:
from functools import partial

tf.random.set_seed(42)
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same",
                        activation="relu", kernel_initializer="he_normal")

model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new = X_test[:10]  # pretend we have new images
y_pred = model.predict(X_new)

Epoch 1/10

1719/1719 [==============================] - 240s 136ms/step - loss: 0.7840 - accuracy: 0.7094 - val_loss: 0.3669 - val_accuracy: 0.8788

Epoch 2/10

1719/1719 [==============================] - 229s 133ms/step - loss: 0.4747 - accuracy: 0.8292 - val_loss: 0.2914 - val_accuracy: 0.8920

Epoch 3/10

1719/1719 [==============================] - 226s 132ms/step - loss: 0.3912 - accuracy: 0.8591 - val_loss: 0.2578 - val_accuracy: 0.9042

Epoch 4/10

1719/1719 [==============================] - 226s 132ms/step - loss: 0.3450 - accuracy: 0.8765 - val_loss: 0.2484 - val_accuracy: 0.9084

Epoch 5/10

1719/1719 [==============================] - 230s 134ms/step - loss: 0.3077 - accuracy: 0.8894 - val_loss: 0.2387 - val_accuracy: 0.9096

Epoch 6/10

1719/1719 [==============================] - 231s 134ms/step - loss: 0.2845 - accuracy: 0.8979 - val_loss: 0.2483 - val_accuracy: 0.9092
Epoch 7/10

1719/1719 [==============================] - 230s 134ms/step - loss: 0.2568 - accuracy: 0.9071 - val_loss: 0.2445 - val_accuracy: 0.9184

Epoch 8/10

1719/1719 [==============================] - 233s 136ms/step - loss: 0.2368 - accuracy: 0.9129 - val_loss: 0.2285 - val_accuracy: 0.9186

Epoch 9/10

1719/1719 [==============================] - 228s 133ms/step - loss: 0.2233 - accuracy: 0.9188 - val_loss: 0.2314 - val_accuracy: 0.9162

Epoch 10/10

1719/1719 [==============================] - 232s 135ms/step - loss: 0.2075 - accuracy: 0.9234 - val_loss: 0.2334 - val_accuracy: 0.9182
313/313 [==============================] - 11s 37ms/step - loss: 0.2620 - accuracy: 0.9158
1/1 [==============================] - 0s 142ms/step